In [1]:
# Dependencies
import requests
import json
from config import tmdb_key, omdb_api_key_1_yp, omdb_api_key_2_mt, omdb_api_key_3_al, omdb_api_key_4_aj
from pprint import pprint
import pandas as pd
from pathlib import Path
import numpy as np


In [2]:
# Set the number of results per page and the total number of desired results
total_result_per_page = 20
total_wanted_results = 100
# Calculate the total number of pages needed to retrieve the desired results
total_pages = int(total_wanted_results / total_result_per_page) #50
current_page = 1

# Function to fetch data from a specific page using the API
def fetch_page(page_number):
        # Construct the URL with the page number to fetch data from
    url = f"https://api.themoviedb.org/3/discover/movie?&include_video=false&language=en-US&sort_by=popularity.desc&page={page_number}&region=US&with_origin_country=US"
     # Set the required headers for the API request
    headers = {
        "accept": "application/json",
        "Authorization": tmdb_key
    }
    # Make a GET request to the API and retrieve the data in JSON format
    response = requests.get(url, headers=headers).json()
    # Return the results from the response
    return response["results"]

# Initialize an empty list to store all retrieved movies
all_movies = []
#Loop through the range of page numbers to fetch data from each page
for page_number in range(1, total_pages + 1):
    all_movies += fetch_page(page_number)

#Print the total number of movies retrieved
print(len(all_movies))    

#Please uncomment the following if you'd like to check the json outcome
# print(len(response["results"]))
# print(json.dumps(response, indent=4, sort_keys=True))

100


In [3]:
#Here we created this code to split the all_movies variable call the omdb api 4 times since it has a limitation of 1000 calls per day.

factor = int(len(all_movies) / 4)
#splitted data 
all_movies_splited_0 = all_movies[0:factor] 
all_movies_splited_1 = all_movies[factor:factor * 2]
all_movies_splited_2 = all_movies[factor * 2: factor * 3]
all_movies_splited_3 = all_movies[factor * 3: factor * 4]


In [4]:
# # #collect titles for omdb fetch
# imdb_titles = []
# for imdb_id in all_movies:
#     imdb_titles.append(imdb_id['title'])
# print(len(all_movies))
# imdb_titles 
# # pprint(all_movies[0])

In [4]:
# Function to fetch details of a specific movie using the API, from this details we wanted the finantial information
def fetch_movie_details(movie_id):
    # Construct the URL with the movie ID to fetch details
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    headers = {
        "accept": "application/json",
        "Authorization": tmdb_key
    }
    # Make a GET request to the API and retrieve the movie details in JSON format
    response = requests.get(url, headers=headers).json()
    # Return the movie details from the response
    return response

In [5]:
# Function to fetch omdb data 
def fetch_movie_omdb(movie_title, omdb_api_key):
    url = "http://www.omdbapi.com/"
    # print(url + movie_title)
    # Make a request to the OMDb API for each movie in the list.
    params = {
        't': movie_title,
        'apikey': omdb_api_key
    }
    response = requests.get(url, params=params)
    print(response.url)
    omdb_data = response.json()
    # Return the omdb data from the response
    return omdb_data

# http://www.omdbapi.com/?i=tt3896198&apikey=cb04c8ea
# http://www.omdbapi.com/?apikey=cb04c8ea&t=Borderlands
# http://www.omdbapi.com/?t=Borderlands&apikey=cb04c8ea

In [6]:
# elements = [all_movies_splited_0,all_movies_splited_1,all_movies_splited_2,all_movies_splited_3]
# api_key_elements = [omdb_api_key_1_yp, omdb_api_key_2_mt, omdb_api_key_3_al, omdb_api_key_4_aj]

elements = [
    {'api_key': omdb_api_key_1_yp, "movies": all_movies_splited_0},
    {'api_key': omdb_api_key_2_mt, "movies": all_movies_splited_1},
    {'api_key': omdb_api_key_3_al, "movies": all_movies_splited_2},
    {'api_key': omdb_api_key_4_aj, "movies": all_movies_splited_3}
]

for element in elements:
    # Iterate through each movie in the list of all movies
    for movie in element['movies']:
        # Fetch details of the current movie using its ID
        movie_details = fetch_movie_details(movie["id"])
        ## Fetch additional movie details(ratings) from the OMDb API using the movie title
        movie_omdb = fetch_movie_omdb(movie['title'], element['api_key'])
        # Update the movie dictionary with the fetched movie details
        movie.update(movie_details)
        movie.update(movie_omdb)

        #Create a key,value pair to determine if movie is financially successful based on revenue and budget comparison
        movie["financial_success"] = movie["revenue"] > movie["budget"]


http://www.omdbapi.com/?t=Borderlands&apikey=225c2e99
http://www.omdbapi.com/?t=Deadpool+%26+Wolverine&apikey=225c2e99
http://www.omdbapi.com/?t=Beetlejuice+Beetlejuice&apikey=225c2e99
http://www.omdbapi.com/?t=Inside+Out+2&apikey=225c2e99
http://www.omdbapi.com/?t=The+Killer&apikey=225c2e99
http://www.omdbapi.com/?t=Despicable+Me+4&apikey=225c2e99
http://www.omdbapi.com/?t=Bad+Boys%3A+Ride+or+Die&apikey=225c2e99
http://www.omdbapi.com/?t=Beetlejuice&apikey=225c2e99
http://www.omdbapi.com/?t=Twisters&apikey=225c2e99
http://www.omdbapi.com/?t=Trap&apikey=225c2e99
http://www.omdbapi.com/?t=Gunner&apikey=225c2e99
http://www.omdbapi.com/?t=The+Union&apikey=225c2e99
http://www.omdbapi.com/?t=Alien%3A+Romulus&apikey=225c2e99
http://www.omdbapi.com/?t=It+Ends+with+Us&apikey=225c2e99
http://www.omdbapi.com/?t=The+Deliverance&apikey=225c2e99
http://www.omdbapi.com/?t=Jackpot%21&apikey=225c2e99
http://www.omdbapi.com/?t=Rebel+Ridge&apikey=225c2e99
http://www.omdbapi.com/?t=Darkness+of+Man&apikey

In [7]:
# Extract rating data one layer up
imdb_rating_name = "Internet Movie Database"
rotten_tomatoes_rating_name = "Rotten Tomatoes"
metacritic_rating_name = 'Metacritic'

for element in elements:
    # Search for ratings
    for movie in element['movies']:
        # Check if "Ratings" key exists
        ratings = movie.get("Ratings", [])
        
        for rating in ratings:
            if imdb_rating_name == rating["Source"]:
                movie["imdb_rating"] = float(rating["Value"].split("/")[0])
            elif rotten_tomatoes_rating_name == rating["Source"]:
                movie["rotten_tomatoes_rating"] = float(rating["Value"].split("%")[0])
            elif metacritic_rating_name == rating["Source"]:
                movie["metacritic_rating"] = float(rating["Value"].split("/")[0])


In [8]:
# checking how many movies we have per group / element
for element in elements:
    print(len(element['movies']))

25
25
25
25


In [12]:
# clear out movies without ratings

# this function filter each movie based in our desired rule
def filter_movies_ratings(movie):
    # here our rule is defined: a movie is only valid for the filter if it has ratings
    return True if len(movie.get("Ratings", [])) > 0 else False

# we go through our groups of movies / elements
for element in elements:
    movies = element['movies']
    # filter and assign/replace the filtered movies
    element['movies'] = list(filter(filter_movies_ratings, movies))

In [10]:
# audit over movies to notice the amount that is left after filtering out the ones that didn't meat the valid criteria.
for element in elements:
    print(len(element['movies']))

23
23
24
24


In [13]:
#Pretty printing our first outcome 
pprint(all_movies_splited_0[1])

{'Actors': 'Ryan Reynolds, Hugh Jackman, Emma Corrin',
 'Awards': '4 wins & 1 nomination',
 'BoxOffice': '$605,188,101',
 'Country': 'United States, United Kingdom, Australia, New Zealand, Canada',
 'DVD': 'N/A',
 'Director': 'Shawn Levy',
 'Genre': 'Action, Adventure, Comedy',
 'Language': 'English, French, Spanish',
 'Metascore': 'N/A',
 'Plot': 'Deadpool is offered a place in the Marvel Cinematic Universe by the '
         'Time Variance Authority, but instead recruits a variant of Wolverine '
         'to save his universe from extinction.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmQxZWM5MzgtY2EzZC00OGUxLWE0Y2EtMDIwOTFlNmQ5MWMyXkEyXkFqcGc@._V1_SX300.jpg',
 'Production': 'N/A',
 'Rated': 'R',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
             {'Source': 'Rotten Tomatoes', 'Value': '78%'}],
 'Released': '26 Jul 2024',
 'Response': 'True',
 'Runtime': '128 min',
 'Title': 'Deadpool & Wolverine',
 'Type': 'movie',
 'Website': 'N/A',
 'Write

In [197]:
#Creating a dataframe
def create_dataframe(all_movies_data):
    df = pd.DataFrame(all_movies_data)
    print(df.info())
    return df

all_movies_df_0 = create_dataframe(all_movies_splited_0)
all_movies_df_1 = create_dataframe(all_movies_splited_1)
all_movies_df_2 = create_dataframe(all_movies_splited_2)
all_movies_df_3 = create_dataframe(all_movies_splited_3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 56 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   adult                   25 non-null     bool   
 1   backdrop_path           25 non-null     object 
 2   genre_ids               25 non-null     object 
 3   id                      25 non-null     int64  
 4   original_language       25 non-null     object 
 5   original_title          25 non-null     object 
 6   overview                25 non-null     object 
 7   popularity              25 non-null     float64
 8   poster_path             25 non-null     object 
 9   release_date            25 non-null     object 
 10  title                   25 non-null     object 
 11  video                   25 non-null     bool   
 12  vote_average            25 non-null     float64
 13  vote_count              25 non-null     int64  
 14  belongs_to_collection   7 non-null      obje

In [198]:
#Displaying all columns 
print(all_movies_df_3.columns)

Index(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'title', 'video', 'vote_average', 'vote_count',
       'belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id',
       'origin_country', 'production_companies', 'production_countries',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'Title',
       'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer',
       'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings',
       'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD',
       'BoxOffice', 'Production', 'Website', 'Response', 'financial_success',
       'imdb_rating', 'rotten_tomatoes_rating', 'metacritic_rating'],
      dtype='object')


In [199]:
#creating a function to selec specific columns from the dataframe
# Selecting specific columns from the DataFrame
def select_specific_columns(dataframe):
    dataframe = dataframe[["imdb_id", "title","release_date", "Runtime", "Genre","overview","Director", "Actors", "Rated","imdbVotes", "popularity", "imdb_rating","rotten_tomatoes_rating", "metacritic_rating", "budget", "revenue", "financial_success"]]
    # Drop rows with any missing values
    dataframe.dropna(inplace=True)
    # Display the length of the DataFrame after dropping rows with missing values
    print(dataframe.columns)
    return dataframe


In [200]:
#Putting previous function into action
# dataframes = [all_movies_df_0,all_movies_df_1,all_movies_df_2,all_movies_df_3]

# for dataframe in dataframes:
    # select_specific_columns(dataframe)
    # display(dataframe.head())
all_movies_df_0 = select_specific_columns(all_movies_df_0)
all_movies_df_1 = select_specific_columns(all_movies_df_1)
all_movies_df_2 = select_specific_columns(all_movies_df_2)
all_movies_df_3 = select_specific_columns(all_movies_df_3)

print(all_movies_df_3.columns)

Index(['imdb_id', 'title', 'release_date', 'Runtime', 'Genre', 'overview',
       'Director', 'Actors', 'Rated', 'imdbVotes', 'popularity', 'imdb_rating',
       'rotten_tomatoes_rating', 'metacritic_rating', 'budget', 'revenue',
       'financial_success'],
      dtype='object')
Index(['imdb_id', 'title', 'release_date', 'Runtime', 'Genre', 'overview',
       'Director', 'Actors', 'Rated', 'imdbVotes', 'popularity', 'imdb_rating',
       'rotten_tomatoes_rating', 'metacritic_rating', 'budget', 'revenue',
       'financial_success'],
      dtype='object')
Index(['imdb_id', 'title', 'release_date', 'Runtime', 'Genre', 'overview',
       'Director', 'Actors', 'Rated', 'imdbVotes', 'popularity', 'imdb_rating',
       'rotten_tomatoes_rating', 'metacritic_rating', 'budget', 'revenue',
       'financial_success'],
      dtype='object')
Index(['imdb_id', 'title', 'release_date', 'Runtime', 'Genre', 'overview',
       'Director', 'Actors', 'Rated', 'imdbVotes', 'popularity', 'imdb_rating',
  

/var/folders/fv/ltttqpzd0bn28sqs0_gjbh300000gn/T/ipykernel_92407/1158900747.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.dropna(inplace=True)
/var/folders/fv/ltttqpzd0bn28sqs0_gjbh300000gn/T/ipykernel_92407/1158900747.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.dropna(inplace=True)
/var/folders/fv/ltttqpzd0bn28sqs0_gjbh300000gn/T/ipykernel_92407/1158900747.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [201]:
#splitting some columns 
def splitting_columns(dataframe):
    dataframe[['star_1','star_2','star_3']] = dataframe['Actors'].str.split(',', n=2, expand=True)
    dataframe[['genre_1','genre_2','genre_3']] = dataframe['Genre'].str.split(',', n=2, expand=True)
    # Split the Director column
    split_directors = dataframe['Director'].str.split(',', n=1, expand=True)

    # Create new columns with default values
    dataframe['director_1'] = split_directors[0]  # This will always exist
    dataframe['director_2'] = ''  # Initialize with empty strings or NaN

    # If there is a second column, assign it to director_2
    if split_directors.shape[1] > 1:
        dataframe['director_2'] = split_directors[1]

    return(dataframe)

In [202]:

#Putting previous function into action
all_movies_df_0 = splitting_columns(all_movies_df_0)
all_movies_df_1 = splitting_columns(all_movies_df_1)
all_movies_df_2 = splitting_columns(all_movies_df_2)
all_movies_df_3 = splitting_columns(all_movies_df_3)

# all_movies_df_0['Director'].str.split(',', n=1, expand=True)
all_movies_df_1.head()

,imdb_id,title,release_date,Runtime,Genre,overview,Director,Actors,Rated,imdbVotes,...,revenue,financial_success,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
5,tt0295701,xXx,2002-08-09,124 min,"Action, Adventure, Thriller",Xander Cage is your standard adrenaline junkie...,Rob Cohen,"Vin Diesel, Asia Argento, Marton Csokas",PG-13,"187,525",...,277448382,True,Vin Diesel,Asia Argento,Marton Csokas,Action,Adventure,Thriller,Rob Cohen,None
12,tt4154756,Avengers: Infinity War,2018-04-25,149 min,"Action, Adventure, Sci-Fi",As the Avengers and their allies have continue...,"Anthony Russo, Joe Russo","Robert Downey Jr., Chris Hemsworth, Mark Ruffalo",PG-13,"1,226,533",...,2052415039,True,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Action,Adventure,Sci-Fi,Anthony Russo,Joe Russo
13,tt23778968,Thelma,2024-06-21,116 min,"Drama, Fantasy, Horror",When 93-year-old Thelma Post gets duped by a p...,Joachim Trier,"Eili Harboe, Kaya Wilkins, Henrik Rafaelsen",Not Rated,"37,145",...,9818454,True,Eili Harboe,Kaya Wilkins,Henrik Rafaelsen,Drama,Fantasy,Horror,Joachim Trier,None
18,tt0078748,Alien,1979-05-25,117 min,"Horror, Sci-Fi","During its return to the earth, commercial spa...",Ridley Scott,"Sigourney Weaver, Tom Skerritt, John Hurt",R,"964,356",...,104931801,True,Sigourney Weaver,Tom Skerritt,John Hurt,Horror,Sci-Fi,None,Ridley Scott,None
19,tt27534307,Speak No Evil,2024-09-11,97 min,"Drama, Horror, Thriller",When an American family is invited to spend th...,Christian Tafdrup,"Morten Burian, Sidsel Siem Koch, Fedja van Huêt",Not Rated,"42,374",...,0,False,Morten Burian,Sidsel Siem Koch,Fedja van Huêt,Drama,Horror,Thriller,Christian Tafdrup,None


In [203]:
# Rename the following columns to keep the same format
all_movies_df_0 = all_movies_df_0.rename(columns={ "Runtime": "runtime", "Genre": "genre","Director": "director", "Actors": "actors", "Rated": "rated", "imdbVotes": "imdb_votes"})
all_movies_df_1 = all_movies_df_1.rename(columns={ "Runtime": "runtime", "Genre": "genre","Director": "director", "Actors": "actors", "Rated": "rated", "imdbVotes": "imdb_votes"})
all_movies_df_2 = all_movies_df_2.rename(columns={ "Runtime": "runtime", "Genre": "genre","Director": "director", "Actors": "actors", "Rated": "rated", "imdbVotes": "imdb_votes"})
all_movies_df_3 = all_movies_df_3.rename(columns={ "Runtime": "runtime", "Genre": "genre","Director": "director", "Actors": "actors", "Rated": "rated", "imdbVotes": "imdb_votes"})
all_movies_df_0    

,imdb_id,title,release_date,runtime,genre,overview,director,actors,rated,imdb_votes,...,revenue,financial_success,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2
7,tt0094721,Beetlejuice,1988-03-30,92 min,"Comedy, Fantasy",A newly dead New England couple seeks help fro...,Tim Burton,"Alec Baldwin, Geena Davis, Michael Keaton",PG,"348,874",...,74849333,True,Alec Baldwin,Geena Davis,Michael Keaton,Comedy,Fantasy,None,Tim Burton,
24,tt27682129,Prey,2024-03-15,100 min,"Action, Adventure, Horror",A young couple is compelled to leave their Chr...,Dan Trachtenberg,"Amber Midthunder, Dakota Beavers, Dane DiLiegro",R,"233,550",...,0,False,Amber Midthunder,Dakota Beavers,Dane DiLiegro,Action,Adventure,Horror,Dan Trachtenberg,


In [204]:
# Format the release_date column to datetime forma
dataframes = [all_movies_df_0,all_movies_df_1,all_movies_df_2,all_movies_df_3]

for dataframe in dataframes:
    dataframe['release_year'] = pd.to_datetime(dataframe['release_date']).dt.year
    dataframe['imdb_rating'] = dataframe['imdb_rating'].astype('float')
    dataframe['rotten_tomatoes_rating'] = dataframe['rotten_tomatoes_rating'].astype('float')
    dataframe['metacritic_rating'] = dataframe['metacritic_rating'].astype('float')
    dataframe['financial_success'] = dataframe['financial_success'].astype('bool')

all_movies_df_2.head()

,imdb_id,title,release_date,runtime,genre,overview,director,actors,rated,imdb_votes,...,financial_success,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2,release_year
0,tt0327597,Coraline,2009-02-05,100 min,"Animation, Drama, Family",Wandering her rambling old house in her boring...,Henry Selick,"Dakota Fanning, Teri Hatcher, John Hodgman",PG,"269,771",...,True,Dakota Fanning,Teri Hatcher,John Hodgman,Animation,Drama,Family,Henry Selick,None,2009
5,tt9362722,Spider-Man: Across the Spider-Verse,2023-05-31,140 min,"Animation, Action, Adventure","After reuniting with Gwen Stacy, Brooklyn’s fu...","Joaquim Dos Santos, Kemp Powers, Justin K. Tho...","Shameik Moore, Hailee Steinfeld, Brian Tyree H...",PG,"394,549",...,True,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Animation,Action,Adventure,Joaquim Dos Santos,"Kemp Powers, Justin K. Thompson",2023
8,tt2316204,Alien: Covenant,2017-05-09,122 min,"Horror, Sci-Fi, Thriller","The crew of the colony ship Covenant, bound fo...",Ridley Scott,"Michael Fassbender, Katherine Waterston, Billy...",R,"311,682",...,True,Michael Fassbender,Katherine Waterston,Billy Crudup,Horror,Sci-Fi,Thriller,Ridley Scott,None,2017
9,tt10872600,Spider-Man: No Way Home,2021-12-15,148 min,"Action, Adventure, Fantasy",Peter Parker is unmasked and no longer able to...,Jon Watts,"Tom Holland, Zendaya, Benedict Cumberbatch",PG-13,"901,293",...,True,Tom Holland,Zendaya,Benedict Cumberbatch,Action,Adventure,Fantasy,Jon Watts,None,2021
10,tt5433140,Fast X,2023-05-17,141 min,"Action, Adventure, Crime",Over many missions and against impossible odds...,Louis Leterrier,"Vin Diesel, Michelle Rodriguez, Jason Statham",PG-13,"124,108",...,True,Vin Diesel,Michelle Rodriguez,Jason Statham,Action,Adventure,Crime,Louis Leterrier,None,2023


In [205]:
#Dropping unnecesary columns
all_movies_df_0.drop(columns=['release_date','genre','director','actors'], inplace=True)
all_movies_df_1.drop(columns=['release_date','genre','director','actors'], inplace=True)
all_movies_df_2.drop(columns=['release_date','genre','director','actors'], inplace=True)
all_movies_df_3.drop(columns=['release_date','genre','director','actors'], inplace=True)

# all_movies_df_2.head()
# all_movies_df_0.head()

In [206]:
all_movies_df_2.dtypes

imdb_id                    object
title                      object
runtime                    object
overview                   object
rated                      object
imdb_votes                 object
popularity                float64
imdb_rating               float64
rotten_tomatoes_rating    float64
metacritic_rating         float64
budget                      int64
revenue                     int64
financial_success            bool
star_1                     object
star_2                     object
star_3                     object
genre_1                    object
genre_2                    object
genre_3                    object
director_1                 object
director_2                 object
release_year                int32
dtype: object

In [207]:
print(all_movies_df_2['financial_success'].dtype)

bool


In [209]:
# outcome
# rules: financial_success == true && imdb_rating > 7 && rotten_tomatoes_rating > 60 && metacritic_rating > 50

# Create a mask for each condition
mask_financial_success = all_movies_df_0['financial_success'] == True
mask_imdb_rating = all_movies_df_0['imdb_rating'] > 6.0
mask_rotten_tomatoes_rating = all_movies_df_0['rotten_tomatoes_rating'] > 60
mask_metacritic_rating = all_movies_df_0['metacritic_rating'] > 60
all_movies_df_0['outcome'] = mask_financial_success & mask_imdb_rating & mask_rotten_tomatoes_rating & mask_metacritic_rating

mask_financial_success = all_movies_df_1['financial_success'] == True
mask_imdb_rating = all_movies_df_1['imdb_rating'] > 6.0
mask_rotten_tomatoes_rating = all_movies_df_1['rotten_tomatoes_rating'] > 60
mask_metacritic_rating = all_movies_df_1['metacritic_rating'] > 60
all_movies_df_1['outcome'] = mask_financial_success & mask_imdb_rating & mask_rotten_tomatoes_rating & mask_metacritic_rating

mask_financial_success = all_movies_df_2['financial_success'] == True
mask_imdb_rating = all_movies_df_2['imdb_rating'] > 6.0
mask_rotten_tomatoes_rating = all_movies_df_2['rotten_tomatoes_rating'] > 60
mask_metacritic_rating = all_movies_df_2['metacritic_rating'] > 60
all_movies_df_2['outcome'] = mask_financial_success & mask_imdb_rating & mask_rotten_tomatoes_rating & mask_metacritic_rating

mask_financial_success = all_movies_df_3['financial_success'] == True
mask_imdb_rating = all_movies_df_3['imdb_rating'] > 6.0
mask_rotten_tomatoes_rating = all_movies_df_3['rotten_tomatoes_rating'] > 60
mask_metacritic_rating = all_movies_df_3['metacritic_rating'] > 60
all_movies_df_3['outcome'] = mask_financial_success & mask_imdb_rating & mask_rotten_tomatoes_rating & mask_metacritic_rating


# mask_financial_success = all_movies_df_2['financial_success'] == True
# mask_imdb_rating = all_movies_df_2['imdb_rating'] > 6.0
# mask_rotten_tomatoes_rating = all_movies_df_2['rotten_tomatoes_rating'] > 60
# mask_metacritic_rating = all_movies_df_2['metacritic_rating'] > 60
# mask_imdb_rating

In [210]:
# all_movies_df_2.drop(columns=['financial_success', 'imdb_rating', 'rotten_tomatoes_rating','metacritic_rating'], inplace=True)
# all_movies_df_2.head()

In [211]:
#display dataframe
all_movies_df_2.head()

,imdb_id,title,runtime,overview,rated,imdb_votes,popularity,imdb_rating,rotten_tomatoes_rating,metacritic_rating,...,star_1,star_2,star_3,genre_1,genre_2,genre_3,director_1,director_2,release_year,outcome
0,tt0327597,Coraline,100 min,Wandering her rambling old house in her boring...,PG,"269,771",211.557,7.8,91.0,80.0,...,Dakota Fanning,Teri Hatcher,John Hodgman,Animation,Drama,Family,Henry Selick,None,2009,True
5,tt9362722,Spider-Man: Across the Spider-Verse,140 min,"After reuniting with Gwen Stacy, Brooklyn’s fu...",PG,"394,549",200.700,8.6,95.0,86.0,...,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Animation,Action,Adventure,Joaquim Dos Santos,"Kemp Powers, Justin K. Thompson",2023,True
8,tt2316204,Alien: Covenant,122 min,"The crew of the colony ship Covenant, bound fo...",R,"311,682",192.510,6.4,65.0,65.0,...,Michael Fassbender,Katherine Waterston,Billy Crudup,Horror,Sci-Fi,Thriller,Ridley Scott,None,2017,True
9,tt10872600,Spider-Man: No Way Home,148 min,Peter Parker is unmasked and no longer able to...,PG-13,"901,293",187.927,8.2,93.0,71.0,...,Tom Holland,Zendaya,Benedict Cumberbatch,Action,Adventure,Fantasy,Jon Watts,None,2021,True
10,tt5433140,Fast X,141 min,Over many missions and against impossible odds...,PG-13,"124,108",183.283,5.7,56.0,56.0,...,Vin Diesel,Michelle Rodriguez,Jason Statham,Action,Adventure,Crime,Louis Leterrier,None,2023,False


In [212]:
#Saving our DataFrame into 4 different csvs.
all_movies_df_0.to_csv("movies_dataset_1.csv", index=False)
all_movies_df_1.to_csv("movies_dataset_1.csv", index=False)
all_movies_df_2.to_csv("movies_dataset_2.csv", index=False)
all_movies_df_3.to_csv("movies_dataset_3.csv", index=False)